# ROUGE implementation in Torch

In [ ]:
---- Have to code up my own split function -- jacked it form StackOverflow
function split(pString)
   local pPattern = " "
   local Table = {}  -- NOTE: use {n = 0} in Lua-5.0
   local fpat = "(.-)" .. pPattern
   local last_end = 1
   local s, e, cap = pString:find(fpat, 1)
   while s do
      if s ~= 1 or cap ~= "" then
     table.insert(Table, cap)
      end
      last_end = e + 1
      s, e, cap = pString:find(fpat, last_end)
   end
   if last_end <= #pString then
      cap = pString:sub(last_end)
      table.insert(Table, cap)
   end
   return Table
end
---- Coding up my own tokenizer
function BuildDic(x)
    local o = {}
    for k, v in pairs(x) do
        if o[v] == nil then
            o[v] = 1
        else
            o[v] = 1 + o[v]
        end
    end
    return o
end
---- Putting them together
function TokenizeData(inputdic)
    local out = {}
    if type(inputdic)=='string' then
        inputdic = split(inputdic)
    end
    for k,v in pairs(inputdic) do
        for j,l in pairs(BuildDic(split(v))) do
           if out[j] == nil then
                out[j] = l
                else 
                out[j] = l + out[j]
            end
        end
    end
    return out
end
--- Now we can calculate ROUGE
function rougeRecall(pred_summary, ref_summaries)
    rsd = TokenizeData(ref_summaries)
    sws = TokenizeData(pred_summary)
    for k,v in pairs(rsd) do
        if sws[k] == nil then
            sws[k] = 0
        end
    end

    for k,v in pairs(sws) do
        if rsd[k] == nil then
            rsd[k] = 0 
        end
    end
    num = 0.
    den = 0.
    for k,v in pairs(rsd) do
        num = num + math.min(rsd[k], sws[k])
        den = den + rsd[k]
    end
    return num/den
end
---- Precision
function rougePrecision(pred_summary, ref_summaries)
    rsd = TokenizeData(ref_summaries)
    sws = TokenizeData(pred_summary)
    for k,v in pairs(rsd) do
        if sws[k] == nil then
            sws[k] = 0
        end
    end
    for k,v in pairs(sws) do
        if rsd[k] == nil then
            rsd[k] = 0
        end
    end
    num = 0.
    den = 0.
    for k,v in pairs(rsd) do
        num = num + math.min(rsd[k], sws[k])
        den = den + sws[k]
    end
    return num/den
end
---- F1
function rougeF1(pred_summary, ref_summaries) 
    rnp = rougeRecall(pred_summary, ref_summaries)
    rnr = rougePrecision(pred_summary, ref_summaries)
    return (2. * rnp * rnr ) / (rnp + rnr)
end

In [ ]:
ref_texts0 = {"summary"}
ref_texts1 = {"true summary"}
ref_texts2 = {"summary", "true summary"}
ref_texts3 = {"summary", "true summary", "my summary"}
summary_pred = "my summary"

In [ ]:
for k,v in pairs({ref_texts0, ref_texts1, ref_texts2, ref_texts3}) do
    print(rougeRecall(summary_pred,v))
end

In [ ]:
for k,v in pairs({ref_texts0, ref_texts1, ref_texts2, ref_texts3}) do
    print(rougePrecision(summary_pred,v))
end

In [ ]:
for k,v in pairs({ref_texts0, ref_texts1, ref_texts2, ref_texts3}) do
    print(rougeF1(summary_pred,v))
end